In [1]:
from jesse import helpers, research

from custom_indicators.all_features import feature_matrix

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2024-01-01"),
    helpers.date_to_timestamp("2024-12-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [5]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

labeler = TripleBarrierLabeler(trading_1m, num_minutes=30)
side_labels = labeler.side_labels()
meta_labels = labeler.meta_labels(side_labels)

2025-02-09 11:38:08.976156 100.0% _apply_pt_sl_on_t1 done after 0.01 minutes. Remaining 0.0 minutes..
2025-02-09 11:38:24.631318 100.0% _apply_pt_sl_on_t1 done after 0.01 minutes. Remaining 0.0 minutes..


In [7]:
candle_df = expand_labels(meta_labels, trading_1m, fill=0)
return_of_label(candle_df)

18.30547767718078

In [9]:
meta_labels

,ret,trgt,bin,side
2024-01-02 00:09:00+00:00,0.003956,0.001074,1,1.0
2024-01-02 00:12:00+00:00,0.003149,0.002175,1,1.0
2024-01-02 00:15:00+00:00,0.004134,0.001990,1,1.0
2024-01-02 00:18:00+00:00,0.004647,0.001778,1,1.0
2024-01-02 00:21:00+00:00,0.004611,0.002921,1,-1.0
...,...,...,...,...
2024-12-30 21:18:00+00:00,0.014374,0.012365,1,-1.0
2024-12-30 21:21:00+00:00,0.013219,0.012245,1,-1.0
2024-12-30 21:24:00+00:00,0.013641,0.012124,1,-1.0
2024-12-30 21:30:00+00:00,0.014594,0.011891,1,-1.0


In [10]:
series_close.join(meta_labels)["bin"].value_counts(dropna=False)

bin
1.0    133752
NaN     41448
Name: count, dtype: int64